In [1]:
# Import minimal requirements
import requests
import json
import re

In [2]:
# Set the base URL for the ARAX reasoner and its endpoint
endpoint_url = 'https://arax.rtx.ai/beta/api/rtx/v1/query'

# Create a dict of the request, specifying the list of DSL commands
query = {"previous_message_processing_plan": {"processing_actions": [
            "add_qnode(curie=DOID:9406, id=n00)",
            "add_qnode(type=chemical_substance, is_set=true, id=n01)",
            "add_qnode(type=protein, id=n02)",
            "add_qedge(source_id=n00, target_id=n01, id=e00)",  
            "add_qedge(source_id=n01, target_id=n02, id=e01)",
            "expand(edge_id=[e00,e01])", 
            "overlay(action=overlay_clinical_info, observed_expected_ratio=true, virtual_edge_type=C1, source_qnode_id=n00, target_qnode_id=n01)", 
            "filter_kg(action=remove_edges_by_attribute, edge_attribute=observed_expected_ratio, direction=below, threshold=3, remove_connected_nodes=t, qnode_id=n01)",   
            "filter_kg(action=remove_orphaned_nodes, node_type=protein)",
            "overlay(action=compute_ngd, virtual_edge_type=N1, source_qnode_id=n01, target_qnode_id=n02)", 
            "filter_kg(action=remove_edges_by_attribute, edge_attribute=ngd, direction=above, threshold=0.85, remove_connected_nodes=t, qnode_id=n02)",
            "resultify(ignore_edge_direction=true)",
            "return(message=true, store=true)"
        ]}}

In [3]:
# Send the request to RTX and check the status
print(f"Executing query at {endpoint_url}\nPlease wait...")
response_content = requests.post(endpoint_url, json=query, headers={'accept': 'application/json'})
status_code = response_content.status_code
if status_code != 200:
    print("ERROR returned with status "+str(status_code))
    print(response_content.json())
else:
    print(f"Response returned with status {status_code}")

Executing query at https://arax.rtx.ai/beta/api/rtx/v1/query
Please wait...
Response returned with status 200


In [4]:
# Unpack respsonse from JSON and display the information log
response_dict = response_content.json()
for message in response_dict['log']:
    if message['level'] >= 20:
        print(message['prefix']+message['message'])

2020-03-14 18:43:01.468568 INFO: ARAXQuery launching
2020-03-14 18:43:01.468590 INFO: Examine input query for needed information for dispatch
2020-03-14 18:43:01.468597 INFO: Found input processing plan. Sending to the ProcessingPlanExecutor
2020-03-14 18:43:01.470696 INFO: Parsing input actions list
2020-03-14 18:43:01.471168 INFO: Adding a QueryNode to Message with parameters {'id': 'n00', 'curie': 'DOID:9406', 'name': None, 'type': None, 'is_set': None}
2020-03-14 18:43:01.471565 INFO: Adding a QueryNode to Message with parameters {'id': 'n01', 'curie': None, 'name': None, 'type': 'chemical_substance', 'is_set': 'true'}
2020-03-14 18:43:01.471685 INFO: Adding a QueryNode to Message with parameters {'id': 'n02', 'curie': None, 'name': None, 'type': 'protein', 'is_set': None}
2020-03-14 18:43:01.471794 INFO: Adding a QueryEdge to Message with parameters {'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': None}
2020-03-14 18:43:01.471827 INFO: Adding a QueryEdge to Message wi

In [5]:
# These URLs provide direct access to resulting data and GUI
if 'id' in response_dict and response_dict['id'] is not None:
    print(f"Data: {response_dict['id']}")
    match = re.search(r'(\d+)$', response_dict['id'])
    if match:
        print(f"GUI: https://arax.rtx.ai/beta/?m={match.group(1)}")
else:
    print("No id was returned in response")

Data: https://arax.rtx.ai/api/rtx/v1/message/1472
GUI: https://arax.rtx.ai/beta/?m=1472


In [6]:
# Or you can view the entire Translator API response Message
print(json.dumps(response_dict, indent=2, sort_keys=True))

{
  "code_description": "Normal completion",
  "context": "https://raw.githubusercontent.com/biolink/biolink-model/master/context.jsonld",
  "datetime": "2020-03-14 18:43:01",
  "id": "https://arax.rtx.ai/api/rtx/v1/message/1472",
  "knowledge_graph": {
    "edges": [
      {
        "id": "3045995",
        "is_defined_by": "RTX",
        "provided_by": "MyChem.info",
        "qedge_id": "e00",
        "relation": "contraindicated_for",
        "source_id": "CHEMBL.COMPOUND:CHEMBL389621",
        "target_id": "DOID:9406",
        "type": "contraindicated_for"
      },
      {
        "id": "3041805",
        "is_defined_by": "RTX",
        "provided_by": "MyChem.info",
        "qedge_id": "e00",
        "relation": "contraindicated_for",
        "source_id": "CHEMBL.COMPOUND:CHEMBL1624",
        "target_id": "DOID:9406",
        "type": "contraindicated_for"
      },
      {
        "id": "418009",
        "is_defined_by": "RTX",
        "provided_by": "MyChem.info",
        "qedge_id